In [1]:
pip install faker psycopg2

     ---------------------------------------- 1.7/1.7 MB 785.2 kB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 348.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [39]:
import psycopg2
from faker import Faker

import json
from datetime import datetime, timedelta
import random

# Membuat Data Dummy

In [40]:
def generate_transaction_code(transaction_type):
  letters = transaction_type.split()[0][:1] + transaction_type.split()[0][2:3]
  random_num = ''.join([str(random.randint(0, 9)) for _ in range(13)])
  return f'{letters} - {random_num}'


In [51]:
def new_dummy_data(num_records):
  fake = Faker()
  data = []

  for _ in range(num_records):
    transaction_type = fake.random_element(elements=('WITHDRAWAL TRANSFER(ATM)',
                                                     'GOPAY', 'DEPOSIT TRANSFER(ATM)',
                                                     'DEPOSIT CROSS BORDER',
                                                     'WITHDRAWAL TRANSFER', 'REVERSAL',
                                                     'BIAYA ADMIN', 'PAYMENT CC',
                                                     'PAYMENT TELCO PASCABAYAR',
                                                     'INTERNATIONAL TRANSACTION',
                                                     'ONLINE SHOPPING', 'PURCHASE',
                                                     'TRANSFER BANK', 'UTILITY PAYMENT'

    ))
    trx_date = datetime.now() - timedelta(days=random.randint(1,365))
    trx_time = f'{random.randint(0, 23):02d}:{random.randint(0, 59):02d}:{random.randint(0, 59):02d}'

    transaction_code = generate_transaction_code(transaction_type)


    record = {"ACCOUNT_NUMBER": fake.random_number(random.randint(6, 6)),
              "TRX_TYPE": fake.random_element(elements=('Ewallet','Credit Cards',
                                                        'Bank Transfers', 'Debit Cards',
                                                        'Cash', 'Checks and Cheques',
                                                        'Electronic Transfers',
                                                        'Cryptocurrencies',
                                                        'Mobile Banking',
                                                        'QR Code Payment', 'Gift Cards')),
              "AMOUNT": fake.random_int(min=-20000000, max=20000000),
              "FLAG_DB_CR": fake.random_element(elements=('C', 'D')),
              "INFORMATION": f'{transaction_type} {transaction_code}',
              "TRX_DATE": int(trx_date.timestamp()*1000),
              "REFERENCE_NUMBER": fake.random_number(digits=6),
              "CURRENCY": "IDR",
              "TRX_TIME": trx_time,
              "CIF": fake.random_number(digits=10),
              "FITUR": fake.random_element(elements=('Gopay', 'OVO', 'Dana',
                                                     'Shopee Pay', 'Paypal',
                                                     'Bank Services'))

    }

    data.append(record)

  return data


In [52]:
num_records = 150000
dummy_data = new_dummy_data(num_records)

# Menyimpan Data Dummy di PostgreSQL

In [53]:
def insert_data_to_postgresql(data):
    connection = psycopg2.connect(
        host="localhost",
        database="db_trans",
        user="postgres",
        password="milo12",
        port='5432'
    )

    cursor = connection.cursor()


    for record in dummy_data:
        insert_query = """
            INSERT INTO user_trans(
                ACCOUNT_NUMBER, TRX_TYPE, AMOUNT, FLAG_DB_CR,
                INFORMATION, TRX_DATE, REFERENCE_NUMBER, CURRENCY,
                TRX_TIME, CIF, FITUR
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            record["ACCOUNT_NUMBER"], record["TRX_TYPE"], record["AMOUNT"],
            record["FLAG_DB_CR"], record["INFORMATION"], 
            datetime.utcfromtimestamp(record["TRX_DATE"] / 1000),
            record["REFERENCE_NUMBER"], record["CURRENCY"], record["TRX_TIME"],
            record["CIF"], record["FITUR"]
        ))

    connection.commit()



In [54]:
insert_data_to_postgresql(dummy_data)